# **CAP 5404 Deep Learning for Computer Graphics**
# *Project II. Neural Networks & Computer Graphics*

Pranath Reddy Kumbam (**UFID**: 8512-0977)



## Part 4: Transfer Learning for Regression 

### Load Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to Working Directory 
%cd drive/My Drive/Acad/DLCG/Project2

/content/drive/My Drive/Acad/DLCG/Project2


In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from torchvision import models
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# Mean Chrominance values
def get_mean_chrominance(img1, img2):
  return (np.mean(img1), np.mean(img2))

# Min-Max Norm 
def norm(images):
    data = []
    for sample in images:
      min = np.amin(sample)
      max = np.amax(sample)
      if min == max:
        continue
      range_val = max - min
      sample = (sample-min)/range_val
      sample = sample.reshape(1,128,128)
      data.append(sample)
    data = np.asarray(data)
    return data

# Import Data
l_train = norm(np.load('./Data/arrays/NCD/L_train.npy'))
a_train = np.load('./Data/arrays/NCD/a_train.npy')/255
b_train = np.load('./Data/arrays/NCD/b_train.npy')/255

l_test = norm(np.load('./Data/arrays/NCD/L_test.npy'))
a_test = np.load('./Data/arrays/NCD/a_test.npy')/255
b_test = np.load('./Data/arrays/NCD/b_test.npy')/255

x_train = l_train
y_train = np.array([get_mean_chrominance(a_train[x], b_train[x]) for x in range(x_train.shape[0])])
y_ts = np.array([get_mean_chrominance(a_test[x], b_test[x]) for x in range(l_test.shape[0])])

x_val = l_test[int(l_test.shape[0]*0.5):]
y_val = y_ts[int(l_test.shape[0]*0.5):]
x_test = l_test[:int(l_test.shape[0]*0.5)]
y_test = y_ts[:int(l_test.shape[0]*0.5)]

batch_size = 100
# Shuffle Data
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_test, y_test = shuffle(x_test, y_test, random_state=0)
x_val, y_val = shuffle(x_val, y_val, random_state=0)

# Split into batches
batch_size = 100
a = 0
b = batch_size
data_temp = []
data_temp2 = []
for i in range(int(x_train.shape[0]/batch_size)):
    data_temp.append(x_train[a:b])
    data_temp2.append(y_train[a:b])
    a += batch_size
    b += batch_size
x_train = np.asarray(data_temp)
y_train = np.asarray(data_temp2)

# Print data shape
print("Data Shape")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

Data Shape
(57, 100, 1, 128, 128)
(57, 100, 2)
(72, 1, 128, 128)
(72, 2)
(73, 1, 128, 128)
(73, 2)


### Define and load pre-trained Models


In [ ]:
# Models
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 64),
            nn.Linear(64, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 64),
            nn.Linear(64, 2),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

# Push models to device and load pre-trained models
model1 = CNN().to("cuda")
model1 = torch.load('./Out/CNN_Regressor_Faces_ReLU.pth')
model2 = CNN2().to("cuda")
model2 = torch.load('./Out/CNN_Regressor_Faces_Tanh.pth')

### Train/Save ReLU Model on NCD and Test 

In [ ]:
# Model
model = model1

# Loss Function
criteria = torch.nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
n_epochs = 1000

# Training
loss_array = []
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y_train[i], dtype=torch.float, device="cuda")
        optimizer.zero_grad()
        outputs = model(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    train_loss_avg = train_loss/x_train.shape[0]
    loss_array.append(train_loss_avg.detach().cpu().numpy())
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  

# Export Training Loss for Plot
np.save('./Out/CNN_Regressor_train_loss_NCD_ReLU.npy', loss_array)

# Export Trained Model for Transfer Learning
torch.save(model, './Out/CNN_Regressor_NCD_ReLU.pth')

# Testing on Test Data
test_data = torch.from_numpy(x_test.astype('float32'))
if torch.cuda.is_available():
  test_data = test_data.cuda()
test_labels = torch.tensor(y_test, dtype=torch.float, device="cuda")
test_outputs = model(test_data)
a_loss = mean_squared_error(test_outputs[:, 0].detach().cpu().numpy(), test_labels[:, 0].cpu().numpy())
b_loss = mean_squared_error(test_outputs[:, 1].detach().cpu().numpy(), test_labels[:, 1].cpu().numpy())
print("Test a Result: " + str(a_loss))
print("Test b Result: " + str(b_loss))

  0%|          | 0/1000 [00:00<?, ?it/s]

Test a Result: 0.0013417687
Test b Result: 0.0009497879


### Train/Save Tanh Model on NCD and Test 

In [ ]:
# Model
model = model2

# Loss Function
criteria = torch.nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
n_epochs = 1000

# Training
loss_array = []
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        if torch.cuda.is_available():
          data = data.cuda()
        # Scale a and b to range [-1,1] as mentioned in the project description 
        labels = torch.tensor(((y_train[i]-0.5)*2), dtype=torch.float, device="cuda")
        optimizer.zero_grad()
        outputs = model(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    train_loss_avg = train_loss/x_train.shape[0]
    loss_array.append(train_loss_avg.detach().cpu().numpy())
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  

# Export Training Loss for Plot
np.save('./Out/CNN_Regressor_train_loss_NCD_Tanh.npy', loss_array)

# Export Trained Model for Transfer Learning
torch.save(model, './Out/CNN_Regressor_NCD_Tanh.pth')

# Testing on Test Data
test_data = torch.from_numpy(x_test.astype('float32'))
if torch.cuda.is_available():
  test_data = test_data.cuda()
test_labels = torch.tensor(((y_test-0.5)*2), dtype=torch.float, device="cuda")
test_outputs = model(test_data)
test_outputs = test_outputs.detach().cpu().numpy()
# Scale it back to [0,1] for testing
test_outputs = (test_outputs/2)+0.5
a_loss = mean_squared_error(test_outputs[:, 0], y_test[:, 0])
b_loss = mean_squared_error(test_outputs[:, 1], y_test[:, 1])
print("Test a Result: " + str(a_loss))
print("Test b Result: " + str(b_loss))

  0%|          | 0/1000 [00:00<?, ?it/s]

Test a Result: 0.0016476523695885005
Test b Result: 0.0009810334768605795
